In [2]:
import pandas as pd
import os
import re

# Lista de universidades y sus respectivos archivos (ajustar la ruta según tus directorios)
universidades = [
    ("Duke", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_Duke.csv"),
    ("Georgia", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_georgia.csv"),
    ("Kansas", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_kansas.csv"),
    ("Kentucky", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_Kentucky.csv"),
    ("LSU", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_lsu.csv"),
    ("Memphis", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_memphis.csv"),
    ("Oklahoma State", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_Okstate.csv"),
    ("Utah", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_utah.csv"),
    ("Washington", r"C:\Users\Yago\Documents\Basketmática\UniProject_Junio2024\Datos CSV\Draft_Picks_From_washington.csv")
]

# Lista para almacenar los dataframes
dataframes = []

# Cargar cada archivo, añadir la columna 'Universidad' y almacenarlo en la lista
for universidad, filepath in universidades:
    df = pd.read_csv(filepath)
    df['Universidad'] = universidad
    dataframes.append(df)

# Preprocesado de datos

In [3]:
# Combinar todos los dataframes en uno solo
df_main = pd.concat(dataframes, ignore_index=True)

# Primer vistazo para comprobar que se han combinado bien y se ha creado la columna 'Universidad'
df_main

,Player,Year,Rd,Overall,From,To,G,MP,FG,FGA,...,PF,PTS,FG%,3P%,FT%,MP/G,PTS/G,TRB/G,AST/G,Universidad
0,Elton Brand,1999,1,1.0,2000.0,2016.0,1058.0,34910.0,6599.0,13185.0,...,3115.0,16827.0,0.500,0.095,0.736,33.0,15.9,8.5,2.1,Duke
1,Grant Hill *,1994,1,3.0,1995.0,2013.0,1026.0,34776.0,6345.0,13126.0,...,2377.0,17137.0,0.483,0.314,0.769,33.9,16.7,6.0,4.1,Duke
2,Mike Dunleavy,2002,1,3.0,2003.0,2017.0,986.0,27340.0,3935.0,8926.0,...,1986.0,11048.0,0.441,0.377,0.803,27.7,11.2,4.3,2.2,Duke
3,Shane Battier,2001,1,6.0,2002.0,2014.0,977.0,30004.0,2982.0,6819.0,...,2208.0,8408.0,0.437,0.384,0.743,30.7,8.6,4.2,1.8,Duke
4,J.J. Redick,2006,1,11.0,2007.0,2021.0,940.0,23964.0,4009.0,8976.0,...,1417.0,12028.0,0.447,0.415,0.892,25.5,12.8,2.0,2.0,Duke
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,Doug Smart,1959,7,50,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington
628,George Spencer,1967,19,161,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington
629,Kim Stewart,1978,6,126,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington
630,Bill Vandenburgh,1949,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Washington


Al haber jugadores que en sus tiempos no se recogían algunas estadísticas, figuran en el conjunto de datos como valores nulos (NA o NaN). Por ello, transformamos dichos valores nulos en '0'.
También es necesario eliminar jugadores drafteados que no llegaron a jugar ningún partido

In [129]:
# Lista de métricas a igualar a 0
list_columns = ['PTS', 'FGA', 'FG', 'FT', 'FTA', 'TRB', 'ORB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
df_main[list_columns] = df_main[list_columns].fillna(0)

# Eliminamos jugadores que fueron drafteados pero no llegaron a jugar ningún partido (partidos y MP son NaN)
df_main.dropna(subset=['G', 'MP'], inplace=True)
df_main.reset_index(inplace=True)
df_main.drop(columns = 'index', inplace=True)

# Hacemos limpieza de algunas de las columnas que tienen valor doble (dos años separados por una coma)
columns_to_clean = ['Year', 'Rd', 'Overall']
df_main[columns_to_clean] = df_main[columns_to_clean].astype(str).replace(r',.*$', '', regex=True)

In [130]:
# Vistazo para ver que se haya hecho la limpieza correctamente
df_main

,Player,Year,Rd,Overall,From,To,G,MP,FG,FGA,...,PF,PTS,FG%,3P%,FT%,MP/G,PTS/G,TRB/G,AST/G,Universidad
0,Elton Brand,1999,1,1.0,2000.0,2016.0,1058.0,34910.0,6599.0,13185.0,...,3115.0,16827.0,0.500,0.095,0.736,33.0,15.9,8.5,2.1,Duke
1,Grant Hill *,1994,1,3.0,1995.0,2013.0,1026.0,34776.0,6345.0,13126.0,...,2377.0,17137.0,0.483,0.314,0.769,33.9,16.7,6.0,4.1,Duke
2,Mike Dunleavy,2002,1,3.0,2003.0,2017.0,986.0,27340.0,3935.0,8926.0,...,1986.0,11048.0,0.441,0.377,0.803,27.7,11.2,4.3,2.2,Duke
3,Shane Battier,2001,1,6.0,2002.0,2014.0,977.0,30004.0,2982.0,6819.0,...,2208.0,8408.0,0.437,0.384,0.743,30.7,8.6,4.2,1.8,Duke
4,J.J. Redick,2006,1,11.0,2007.0,2021.0,940.0,23964.0,4009.0,8976.0,...,1417.0,12028.0,0.447,0.415,0.892,25.5,12.8,2.0,2.0,Duke
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,Bobby Jones,2006,2,37,2007.0,2008.0,91.0,867.0,103.0,233.0,...,144.0,288.0,0.442,0.306,0.685,9.5,3.2,1.6,0.5,Washington
389,C.J. Wilcox,2014,1,28,2015.0,2017.0,66.0,376.0,50.0,135.0,...,30.0,132.0,0.370,0.333,0.813,5.7,2.0,0.5,0.5,Washington
390,Rich Manning,1993,2,40,1996.0,1997.0,55.0,512.0,81.0,195.0,...,66.0,182.0,0.415,0.400,0.643,9.3,3.3,1.7,0.2,Washington
391,Lars Hansen,1976,3,37,1979.0,1979.0,30.0,410.0,58.0,114.0,...,56.0,152.0,0.509,NaN,0.581,13.7,5.1,3.9,0.9,Washington


Una vez ajusadas estas incongruencias, toca preparar un diccionario que irá almacenando los puntos conseguidos por cada universidad, y calcular algunas métricas que necesitaremos en alguna de las rondas

In [131]:
# Inicializamos los puntos de la clasificación
university_points = {
    'Duke': 0,
    'Georgia': 0,
    'Kansas': 0,
    'Kentucky': 0,
    'LSU': 0,
    'Memphis': 0,
    'Oklahoma State': 0,
    'Utah': 0,
    'Washington': 0
}

# Transformamos gran parte de las columnas en enteros para evitar problemas en las métricas compuestas
list_columns = ['PTS', 'FGA', 'FG', 'FT', 'FTA', 'TRB', 'ORB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'G',  'MP']
df_main[list_columns] = df_main[list_columns].astype(int)

# Creamos la columna valoración media por partido de cara a la tercera ronda
df_main['VAL/G'] = round(
    (df_main['PTS'] + df_main['TRB'] + df_main['AST'] + df_main['STL'] + df_main['BLK'] - df_main['TOV'] - df_main['PF'] - (df_main['FGA'] - df_main['FG']) - (df_main['FTA'] - df_main['FT']))/(df_main['G']), 1)

# Creamos la columna valoración defensiva media por partido de cara a la cuarta ronda
df_main['DVAL/G'] = round(
    (df_main['STL'] + df_main['BLK'] + (df_main['TRB'] - df_main['ORB']))/(df_main['G']), 1)

# Creamos la columna TS% de cara a la quinta ronda
df_main['TS%'] = round(
    df_main['PTS']/(2*(df_main['FGA'] + 0.44*df_main['FTA'])), 3)

# Creamos la columna HOF, una flag que indica si el jugador es Hall of Fame (1) para la última ronda
df_main['HOF'] = df_main['Player'].apply(lambda x: 1 if '*' in x else 0)

In [132]:
# Vamos también a crear un DataFrame por cada universidad por temas de sencillez a la hora de filtrar
df_duke = df_main[df_main['Universidad'] == 'Duke']
df_georgia = df_main[df_main['Universidad'] == 'Georgia']
df_kansas = df_main[df_main['Universidad'] == 'Kansas']
df_kentucky = df_main[df_main['Universidad'] == 'Kentucky']
df_lsu = df_main[df_main['Universidad'] == 'LSU']
df_memphis = df_main[df_main['Universidad'] == 'Memphis']
df_okstate = df_main[df_main['Universidad'] == 'Oklahoma State']
df_utah = df_main[df_main['Universidad'] == 'Utah']
df_washington = df_main[df_main['Universidad'] == 'Washington']

# Diccionario de DataFrames por universidad
df_universities = {
    'Duke': df_duke,
    'Georgia': df_georgia,
    'Kansas': df_kansas,
    'Kentucky': df_kentucky,
    'LSU': df_lsu,
    'Memphis': df_memphis,
    'Oklahoma State': df_okstate,
    'Utah': df_utah,
    'Washington': df_washington
}

In [133]:
# Último vistazo antes de empezar las rondas
df_main.head()

,Player,Year,Rd,Overall,From,To,G,MP,FG,FGA,...,FT%,MP/G,PTS/G,TRB/G,AST/G,Universidad,VAL/G,DVAL/G,TS%,HOF
0,Elton Brand,1999,1,1.0,2000.0,2016.0,1058,34910,6599,13185,...,0.736,33.0,15.9,8.5,2.1,Duke,16.8,8.0,0.548,0
1,Grant Hill *,1994,1,3.0,1995.0,2013.0,1026,34776,6345,13126,...,0.769,33.9,16.7,6.0,4.1,Duke,16.1,6.6,0.551,1
2,Mike Dunleavy,2002,1,3.0,2003.0,2017.0,986,27340,3935,8926,...,0.803,27.7,11.2,4.3,2.2,Duke,9.8,4.6,0.555,0
3,Shane Battier,2001,1,6.0,2002.0,2014.0,977,30004,2982,6819,...,0.743,30.7,8.6,4.2,1.8,Duke,9.0,4.8,0.559,0
4,J.J. Redick,2006,1,11.0,2007.0,2021.0,940,23964,4009,8976,...,0.892,25.5,12.8,2.0,2.0,Duke,9.2,2.4,0.602,0


# Ronda 1: Jugadores drafteados en 1a ronda

In [134]:
# Lista para insertar los valores para cada universidad
first_round_picks = []

# Contamos los jugadores drafteados en primera ronda para cada universidad
for university, df in df_universities.items():
    first_round_count = len(df[df['Rd'] == '1'])
    first_round_picks.append([university, first_round_count])

# Ordenamos la lista según el segundo valor de cada lista anidada, de mayor a menor para ver los mejores equipos primero
first_round_picks.sort(key=lambda x: x[1], reverse=True)
print(first_round_picks)

# Lista de puntos asignados
points = [9, 8, 7, 6, 5, 5, 3, 2, 1]

# Repartimos los puntos según la clasificación
for i, (university, _) in enumerate(first_round_picks):
    university_points[university] += points[i]

# Imprimimos los puntos de cada universidad
print(university_points)

[['Kentucky', 55], ['Duke', 53], ['Kansas', 36], ['Washington', 17], ['LSU', 16], ['Memphis', 16], ['Utah', 13], ['Oklahoma State', 10], ['Georgia', 8]]
{'Duke': 8, 'Georgia': 1, 'Kansas': 7, 'Kentucky': 9, 'LSU': 5, 'Memphis': 5, 'Oklahoma State': 2, 'Utah': 3, 'Washington': 6}


# Ronda 2: Minutos totales disputados de los jugadores drafteados

In [135]:
# Creamos una lista para insertar los valores para cada universidad
minutes_university = []

# Contamos los minutos totales de todos los jugadores de cada universidad
for university, df in df_universities.items():
    minutes_count = df['MP'].sum()
    minutes_university.append([university, minutes_count])

# Ordenamos la lista según el segundo valor de cada lista anidada, de mayor a menor para ver los mejores equipos primero
minutes_university.sort(key=lambda x: x[1], reverse=True)
print(minutes_university)

# Lista de puntos asignados
points = [9, 8, 7, 6, 5, 4, 3, 2, 1]

# Repartimos los puntos según la clasificación
for i, (university, _) in enumerate(minutes_university):
    university_points[university] += points[i]

# Imprimimos los puntos de cada universidad
print(university_points)

[['Kentucky', 854425], ['Duke', 730902], ['Kansas', 601187], ['Washington', 252044], ['Memphis', 236282], ['LSU', 235796], ['Utah', 222102], ['Georgia', 164742], ['Oklahoma State', 95886]]
{'Duke': 16, 'Georgia': 3, 'Kansas': 14, 'Kentucky': 18, 'LSU': 9, 'Memphis': 10, 'Oklahoma State': 3, 'Utah': 6, 'Washington': 12}


# Ronda 3: Valoración media de los jugadores drafteados

In [136]:
# Creamos una lista para insertar los valores para cada universidad
valoration_university = []

# Calculamos la valoración media de todos los jugadores de cada universidad
for university, df in df_universities.items():
    mean_val = round(df['VAL/G'].sum()/len(df), 2)
    valoration_university.append([university, mean_val])

# Ordenamos la lista según el segundo valor de cada lista anidada, de mayor a menor para ver los mejores equipos primero
valoration_university.sort(key=lambda x: x[1], reverse=True)
print(valoration_university)

# Lista de puntos asignados
points = [9, 8, 7, 6, 5, 4, 3, 2, 1]

# Repartimos los puntos según la clasificación
for i, (university, _) in enumerate(valoration_university):
    university_points[university] += points[i]

# Imprimimos los puntos de cada universidad
print(university_points)

[['Duke', 7.93], ['Kentucky', 7.91], ['Washington', 7.2], ['Utah', 6.94], ['LSU', 6.91], ['Kansas', 6.71], ['Memphis', 6.52], ['Georgia', 6.49], ['Oklahoma State', 5.59]]
{'Duke': 25, 'Georgia': 5, 'Kansas': 18, 'Kentucky': 26, 'LSU': 14, 'Memphis': 13, 'Oklahoma State': 4, 'Utah': 12, 'Washington': 19}


# Ronda 4: Valoración defensiva media de los jugadores drafteados

In [137]:
# Creamos una lista para insertar los valores para cada universidad
defval_university = []

# Calculamos la valoración defensiva media de todos los jugadores de cada universidad
for university, df in df_universities.items():
    mean_dval = round(df['DVAL/G'].sum()/len(df), 2)
    defval_university.append([university, mean_dval])

# Ordenamos la lista según el segundo valor de cada lista anidada, de mayor a menor para ver los mejores equipos primero
defval_university.sort(key=lambda x: x[1], reverse=True)
print(defval_university)

# Lista de puntos asignados
points = [9, 8, 7, 6, 5, 4, 3, 2, 1]

# Repartimos los puntos según la clasificación
for i, (university, _) in enumerate(defval_university):
    university_points[university] += points[i]

# Imprimimos los puntos de cada universidad
print(university_points)

[['Kentucky', 3.77], ['Kansas', 3.69], ['Duke', 3.62], ['LSU', 3.58], ['Washington', 3.5], ['Memphis', 3.42], ['Utah', 3.42], ['Oklahoma State', 3.2], ['Georgia', 3.16]]
{'Duke': 32, 'Georgia': 6, 'Kansas': 26, 'Kentucky': 35, 'LSU': 20, 'Memphis': 17, 'Oklahoma State': 6, 'Utah': 15, 'Washington': 24}


# Ronda 5: TS% medio de todos los jugadores drafteados

In [138]:
# Creamos una lista para insertar los valores para cada universidad
ts_university = []

# Calculamos el TS% medio de todos los jugadores de cada universidad
for university, df in df_universities.items():
    mean_ts = round(df['TS%'].sum()/len(df), 4)
    ts_university.append([university, mean_ts])

# Ordenamos la lista según el segundo valor de cada lista anidada, de mayor a menor para ver los mejores equipos primero
ts_university.sort(key=lambda x: x[1], reverse=True)
print(ts_university)

# Lista de puntos asignados
points = [9, 8, 7, 6, 5, 4, 3, 2, 1]

# Repartimos los puntos según la clasificación
for i, (university, _) in enumerate(ts_university):
    university_points[university] += points[i]

# Imprimimos los puntos de cada universidad
print(university_points)

[['Washington', 0.527], ['Duke', 0.5192], ['LSU', 0.5098], ['Kansas', 0.5077], ['Kentucky', 0.5018], ['Memphis', 0.4991], ['Georgia', 0.4986], ['Utah', 0.4933], ['Oklahoma State', 0.4703]]
{'Duke': 40, 'Georgia': 9, 'Kansas': 32, 'Kentucky': 40, 'LSU': 27, 'Memphis': 21, 'Oklahoma State': 7, 'Utah': 17, 'Washington': 33}


# Ronda 6: Jugadores que han entrado en el Hall Of Fame (x2)

In [126]:
# Ronda Final: HOF (CUENTA DOBLE!)

hof_university = []

# Contamos los jugadores que han sido HOF de cada universidad
for university, df in df_universities.items():
    hof_count = len(df[df['HOF'] == 1])
    hof_university.append([university, hof_count])

# Ordenamos la lista según el segundo valor de cada lista anidada, de mayor a menor para ver los mejores equipos primero
hof_university.sort(key=lambda x: x[1], reverse=True)
print(hof_university)

# Lista de puntos asignados (¡cuenta doble! Para los empates distribuimos uniformemente los puntos correspondientes)
points = [18, 16, 14, 10, 10, 10, 4, 4, 4]

# Repartimos los puntos según la clasificación
for i, (university, _) in enumerate(hof_university):
    university_points[university] += points[i]

# Imprimimos los puntos de cada universidad
print(university_points)

[['Kentucky', 5], ['Kansas', 4], ['LSU', 3], ['Duke', 1], ['Georgia', 1], ['Washington', 1], ['Memphis', 0], ['Oklahoma State', 0], ['Utah', 0]]
{'Duke': 50, 'Georgia': 19, 'Kansas': 48, 'Kentucky': 58, 'LSU': 41, 'Memphis': 25, 'Oklahoma State': 11, 'Utah': 21, 'Washington': 43}


Clasificación final:

1. Kentucky: 58 puntos.
2. Duke: 50 puntos.
3. Kansas: 48 puntos.
4. Washington: 43 puntos.
5. LSU: 41 puntos.
6. Memphis: 25 puntos.
7. Utah: 21 puntos.
8. Georgia: 19 puntos.
9. Oklahoma State: 11 puntos.

In [139]:
# Guardamos el DataFrame actualizado en un archivo Excel para luego usarlo en Tableau (ajusar la ruta según tus directorios)
df_main.to_excel(r"C:\Users\Yago\Documents\Basketmática\PicksUniversidades_Julio2024\uniprojects_cleaned.xlsx")